In [2]:
pip install pyspark pandas

You should consider upgrading via the 'C:\Users\s546981\big-data-venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
household = spark.read.format('csv').option('header','true').load('/Users/s546981/OneDrive - nwmissouri.edu/Desktop/DV Project/US_household income.csv')
household.createOrReplaceTempView('ushousehold')
household.show()
display(household)

+--------+----------+----------+--------+----------------+--------------+-----------------+----+-------+--------+---------+---------+---------+---------+----------+-----+------+-----+----------------+
|      id|State_Code|State_Name|State_ab|          County|          City|            Place|Type|Primary|Zip_Code|Area_Code|    ALand|   AWater|      Lat|       Lon| Mean|Median|Stdev|           sum_w|
+--------+----------+----------+--------+----------------+--------------+-----------------+----+-------+--------+---------+---------+---------+---------+----------+-----+------+-----+----------------+
|01011000|        01|   Alabama|      AL|   Mobile County|     Chickasaw|   Chickasaw city|City|  place|   36611|      251| 10894952|   909156| 30.77145|-88.079697|38773| 30506|33101|1638.26051309172|
|01011010|        01|   Alabama|      AL|  Barbour County|    Louisville|        Clio city|City|  place|   36048|      334| 26070325|    23254|31.708516|-85.611039|37725| 19528|43789|258.017684685

DataFrame[id: string, State_Code: string, State_Name: string, State_ab: string, County: string, City: string, Place: string, Type: string, Primary: string, Zip_Code: string, Area_Code: string, ALand: string, AWater: string, Lat: string, Lon: string, Mean: string, Median: string, Stdev: string, sum_w: string]

In [5]:
#Dropping duplicates in the dataframe
if household.count() > household.dropDuplicates(["id"]).count():
    raise ValueError('Data has duplicates')
    

In [6]:
#Changing datatypes into useable format
from pyspark.sql.types import IntegerType,BooleanType,DateType
household = household.withColumn("id",household.id.cast(IntegerType()))
household = household.withColumn("State_Code",household.State_Code.cast(IntegerType()))
household = household.withColumn("ALand",household.ALand.cast(IntegerType()))
household = household.withColumn("AWater",household.AWater.cast(IntegerType()))
household = household.withColumn("Lat",household.Lat.cast(BooleanType()))
household = household.withColumn("Lon",household.Lon.cast(BooleanType()))
household = household.withColumn("Mean",household.Mean.cast(IntegerType()))
household = household.withColumn("Median",household.Median.cast(IntegerType()))
household = household.withColumn("Stdev",household.Stdev.cast(IntegerType()))
household = household.withColumn("sum_w",household.sum_w.cast(BooleanType()))

In [7]:
#After changing the datatype checking the changed datatypes using display
display(household)

DataFrame[id: int, State_Code: int, State_Name: string, State_ab: string, County: string, City: string, Place: string, Type: string, Primary: string, Zip_Code: string, Area_Code: string, ALand: int, AWater: int, Lat: boolean, Lon: boolean, Mean: int, Median: int, Stdev: int, sum_w: boolean]

In [8]:
#1.	Identify county’s which having the mean of household income of the specified geographic location less than or equal to 100
# Finding counties and cities which have household income less than or equal to 100 in all over the dataset.
spark.sql('select County ,City from ushousehold where mean <= 100 and County = "Apache County" ').show()
spark.sql('select County ,City from ushousehold where mean <= 100').show()

+-------------+-----------+
|       County|       City|
+-------------+-----------+
|Apache County|  Ehrenberg|
|Apache County|   Dateland|
|Apache County|Ft Huachuca|
|Apache County|    Phoenix|
|Apache County|    Phoenix|
|Apache County|    Phoenix|
|Apache County|       Yuma|
+-------------+-----------+

+--------------------+--------------+
|              County|          City|
+--------------------+--------------+
|    St. Clair County|       Vincent|
|      Autauga County|    Alexandria|
|      Autauga County|Bayou La Batre|
|      Autauga County|        Mobile|
|Ketchikan Gateway...|     Ketchikan|
|Valdez-Cordova Ce...|    Glennallen|
|       Mohave County| Dolan Springs|
|       Apache County|     Ehrenberg|
|       Apache County|      Dateland|
|       Apache County|   Ft Huachuca|
|       Apache County|       Phoenix|
|       Apache County|       Phoenix|
|       Apache County|       Phoenix|
|       Apache County|          Yuma|
|     Arkansas County|        Conway|
|      

In [9]:
#2.	Find the average of square area of Land and average of square of water in all the states
spark.sql('select avg(ALand), avg(AWater) from ushousehold').show()

+--------------------+-----------------+
|          avg(ALand)|      avg(AWater)|
+--------------------+-----------------+
|1.1658927410942015E8|6952054.178411117|
+--------------------+-----------------+



In [10]:
household.dropna().show(truncate=False)

+---+----------+----------+--------+------+----+-----+----+-------+--------+---------+-----+------+---+---+----+------+-----+-----+
|id |State_Code|State_Name|State_ab|County|City|Place|Type|Primary|Zip_Code|Area_Code|ALand|AWater|Lat|Lon|Mean|Median|Stdev|sum_w|
+---+----------+----------+--------+------+----+-----+----+-------+--------+---------+-----+------+---+---+----+------+-----+-----+
+---+----------+----------+--------+------+----+-----+----+-------+--------+---------+-----+------+---+---+----+------+-----+-----+



In [11]:
#3.	Determine top 5 united states households income and least 5 household income in all the states.
spark.sql('select City,Mean from ushousehold order by Mean Desc LIMIT 5').show()
spark.sql('select City,Mean from ushousehold order by Mean ASC LIMIT 5').show()

+-------------+-----+
|         City| Mean|
+-------------+-----+
|Breinigsville|99994|
|      Lincoln|99974|
|     Pasadena|99951|
|    Eatontown|99944|
|       Elburn|99936|
+-------------+-----+

+---------------+----+
|           City|Mean|
+---------------+----+
|         Laredo|   0|
|Rio Grande City|   0|
|          Heath|   0|
|      Oak Ridge|   0|
|Rio Grande City|   0|
+---------------+----+



In [12]:
# 4.Identify the zip code and area code of least 5 mean and median of household income in all the states.
spark.sql('select Zip_Code, Area_Code from ushousehold order by Mean DESC LIMIT 5').show()
spark.sql('select Zip_Code, Area_Code from ushousehold order by Median DESC LIMIT 5').show()

+--------+---------+
|Zip_Code|Area_Code|
+--------+---------+
|   18031|      610|
|   68506|      402|
|   21122|      410|
|   07724|      732|
|   60119|      630|
+--------+---------+

+--------+---------+
|Zip_Code|Area_Code|
+--------+---------+
|   85255|      602|
|   02170|      617|
|   30066|      770|
|   92806|      714|
|   91745|      626|
+--------+---------+



In [13]:
# 5.	Identify and determine the distinct values of the square area of water and area of land where the place type is city.
spark.sql('select DISTINCT ALAND, AWATER from ushousehold where Type = "City"').show()

+--------+--------+
|   ALAND|  AWATER|
+--------+--------+
| 4985031|       0|
|10047646|       0|
|  921741|       0|
|11803623|    2619|
|42914903|26768761|
|26359013|  603744|
| 1053155|       0|
|  862209|       0|
| 2517563|       0|
| 1985247|     788|
|  240330|       0|
| 2068424|       0|
| 9717030|    2167|
| 7156988|       0|
| 5857953|   45930|
| 1342689| 1311662|
|74913028|  364965|
|15834746|  847182|
|15369718|  289140|
| 2763100|  514531|
+--------+--------+
only showing top 20 rows



In [29]:
#6.	Find the place with the highest mean value in the mean household income column.
spark.sql('select City, Place from ushousehold order by mean DESC').show()

+---------------+---------------+
|           City|          Place|
+---------------+---------------+
|  Breinigsville|  Breinigsville|
|        Lincoln|            Ayr|
|       Pasadena| Little Orleans|
|      Eatontown|   Absecon city|
|         Elburn|         Elburn|
|          Chino|   Alameda city|
|     Northville|   Hubbard Lake|
|          Wayne|   Absecon city|
|      Elk Grove|   Alameda city|
|     Parsippany|   Absecon city|
|    Bridge City|        Elkhart|
|      Fairbanks|    Akutan city|
|     Lewisville|Lewisville city|
|      Vacaville|   Alameda city|
|        Hampton|   Chincoteague|
|  Overland Park|       Gas city|
|         Fresno|   Alameda city|
|      Greenwich|         Bethel|
|      Hawthorne|   Alameda city|
|Clifton Heights|    Abbottstown|
+---------------+---------------+
only showing top 20 rows



In [30]:
#7.	Calculate the total land and water area of all places in Alabama state.
household = household.withColumn("ALand",household.ALand.cast(BooleanType()))
household = household.withColumn("AWater",household.AWater.cast(BooleanType()))
spark.sql('SELECT SUM(ALand) AS TotalLandArea, SUM(AWater) AS TotalWaterArea from ushousehold where State_ab = "AL"').show()

+---------------+--------------+
|  TotalLandArea|TotalWaterArea|
+---------------+--------------+
|5.7292926995E10| 2.417604645E9|
+---------------+--------------+



In [33]:
    #8. 	Calculate the average value in the ‘Stdev’ column for all places in each county in Missouri.
spark.sql('select county , AVG(Stdev) AS AvgStdev from ushousehold where State_ab = "MO" GROUP BY County').show()

+------------------+------------------+
|            county|          AvgStdev|
+------------------+------------------+
|     Benton County|           30310.0|
|       Cass County|           36245.5|
|     Maries County|           21067.0|
|       Clay County|           36896.0|
|    Jackson County|           49017.0|
|  St. Louis County|           49155.5|
|Mississippi County|           11440.0|
|    Audrain County|           27672.0|
|  Gasconade County|           34967.0|
|     DeKalb County|           41228.0|
|     Newton County|52486.666666666664|
|     Greene County|           36935.5|
|    Nodaway County|           35378.0|
|   Moniteau County|           41795.0|
|  Lafayette County|           27579.0|
|       Dade County|           38597.0|
|     Miller County|           24659.5|
|    Carroll County|           33366.0|
|      Osage County|           37477.0|
|      Ralls County|           48407.0|
+------------------+------------------+
only showing top 20 rows



In [36]:
#9.Retrieve the city name, county name, and total land area (in square meters) for all places with a median income between $40,000 and $60,000
spark.sql('SELECT city, county, SUM(ALand) AS total_land_area FROM ushousehold WHERE median BETWEEN 40000 AND 60000 GROUP BY city, county').show()

+-----------------+-----------------+---------------+
|             city|           county|total_land_area|
+-----------------+-----------------+---------------+
|Owens Cross Roads|   Autauga County|   1.61262786E8|
|         Oroville|   Alameda County|    1.8244928E7|
|         Martinez|   Alameda County|    1.7044655E7|
|    Coral Springs|   Alachua County|      7595303.0|
|           Butler|   Appling County|   2.67316168E8|
|  Carpentersville|      Kane County|    2.0445861E7|
|     Mount Morris|     Adams County|   1.26332935E8|
|            Flora|     Adams County|   3.17893925E8|
|    South Holland|     Adams County|      1557571.0|
|        Herington|    Morris County|       224891.0|
|      Loreauville|    Acadia Parish|   1.79117402E8|
|          Lafitte|    Acadia Parish|    6.8775292E7|
|        Cambridge|Barnstable County|      3354699.0|
| Dearborn Heights|    Alcona County|      8737901.0|
|          Mankato|Blue Earth County|    4.9080715E7|
|      Sauk Rapids|    Aitki

In [37]:
#10. Calculate the average median income for each state, ordered from highest to lowest.
spark.sql('SELECT state_name, AVG(median) AS avg_median_income FROM ushousehold GROUP BY state_name ORDER BY avg_median_income DESC').show()

+--------------------+------------------+
|          state_name| avg_median_income|
+--------------------+------------------+
|          New Jersey|125685.82995495496|
|         Connecticut|120557.34929577465|
|       Massachusetts|118008.03880597015|
|            Maryland|113988.23208191126|
|             Wyoming|113605.57333333333|
|                Utah|109378.08045977012|
|              Alaska|108911.69523809524|
|        Rhode Island|105751.98165137615|
|            Virginia|103971.86369426752|
|       New Hampshire|101146.00763358778|
|          California|100581.73292682927|
|            New York|100544.02685185186|
|          Washington| 99918.09156976744|
|            Colorado| 99512.92205323195|
|              Hawaii| 95679.36231884058|
|District of Columbia|       93759.59375|
|            Nebraska| 90510.68727272727|
|              Nevada| 90228.09811320755|
|           Minnesota| 88461.72542901715|
|            Delaware| 87086.03409090909|
+--------------------+------------